In [200]:
from typing import List, Dict
import nixio
from os.path import join
from os import PathLike, listdir
import os
import pandas as pd
import regex as re
import numpy as np

In [201]:
def main():
    for nixdir in nixdirs():
        nixfiles = _find_nix_files(nixdir)
        dfs = []
        for subject, sessions in nixfiles.items():
            for session, path in sessions.items():
                print("Reading subject", subject, "session", session)
                try:
                    nixfile = nixio.File.open(path, nixio.FileMode.ReadOnly)
                    df = _read_session(nixfile, subject, session)
                    dfs.append(df)
                except Exception as e:
                    print("Error reading subject", subject, "session", session)
                    print(e)
        df = pd.concat(dfs)
        nixname = nixdir.split(os.sep)[-2]
        out_file = join("out", f"{nixname}.tsv")

        df.to_csv(out_file, sep="\t", index=False)

In [202]:
def nixdirs() -> List[PathLike]:
    return [_compose_nixdir(subpath) for subpath in _dataset_subpaths()]

In [203]:
def _dataset_subpaths() -> List[PathLike]:
    return [
        join("Human_Amygdala_MUA_sEEG_FearVideo", "data_NIX"),
        join("Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM", "data_NIX"),
        "Human_MTL_units_visual_WM"
    ]

In [204]:
def _compose_nixdir(subpath: PathLike) -> PathLike:
    base_dir = join("/", "mnt", "c", "Users", "conta", "git", "USZ_NCH")
    return join(base_dir, subpath)

In [205]:
def _find_nix_files(nixdir: PathLike) -> Dict[int, Dict[int, PathLike]]:
    nix_files = {}
    for file in listdir(nixdir):
        match = _NIX_REGEX.match(file)
        if match:
            subject, session = map(int, match.groups())
            if subject not in nix_files:
                nix_files[subject] = {}
            nix_files[subject][session] = join(nixdir, file)
    return nix_files

In [206]:
_NIX_REGEX = re.compile(r"Data_Subject_(\d{2})_Session_(\d{2}).h5")

In [207]:
def _read_session(nixfile: nixio.File, subject: int, session: int) -> pd.DataFrame:
    subject_dict = _read_subject(nixfile)
    trials = _read_trials(nixfile)

    subject_df = pd.DataFrame([subject_dict])
    df = pd.concat([subject_df, trials], axis=1)
    df.insert(0, "Subject", subject, allow_duplicates=True)
    df.insert(1, "Session", session, allow_duplicates=True)

    cols = df.select_dtypes(include="number").columns
    df[cols] = df[cols].applymap(np.int64)
    df.reset_index(drop=True, inplace=True)
    return df

In [208]:
def _read_subject(nixfile: nixio.File) -> Dict[str, str]:
    subject = nixfile.sections["Subject"]
    return _read_into_dict(subject.props)

In [209]:
def _read_trials(nixfile: nixio.File) -> pd.DataFrame:
    trials = nixfile.sections["Session"].sections["Trial properties"]
    trial_dicts = [_read_trial(trial) for trial in trials]
    trials_df = pd.DataFrame(trial_dicts)
    if 'Correct' in trials_df.columns:
        return _to_average_correct(trials_df)
    elif 'Condition name' in trials_df.columns:
        return _to_number_of_trials(trials_df)
    else:
        raise ValueError("Unknown trial type")


In [210]:
def _read_trial(trial: nixio.Section) -> Dict[str, str]:
    return _read_into_dict(trial.props)

In [211]:
def _to_average_correct(trials: pd.DataFrame) -> pd.DataFrame:
    correct = trials[trials["Correct"] == 1]
    total = len(trials)
    percent_correct = round((len(correct) / total) * 100)
    return pd.DataFrame([{"Percent correct": percent_correct, "Total trials": total}])

In [212]:
def _to_number_of_trials(trials: pd.DataFrame) -> pd.DataFrame:
    trial_count = len(trials)
    return pd.DataFrame([{"Total trials": trial_count}])

In [213]:
def _read_into_dict(iterable) -> Dict[str, str]:
    props_dict = {}
    for item in iterable:
        name = item.name
        value = item.values[0]
        props_dict[name] = value

    return props_dict

In [214]:

if __name__ == "__main__":
    main()

Reading subject 1 session 1
Reading subject 2 session 1
Reading subject 3 session 1
Reading subject 4 session 1
Reading subject 5 session 1
Reading subject 6 session 1
Reading subject 7 session 1
Reading subject 8 session 1
Reading subject 9 session 1
Reading subject 1 session 1
Reading subject 1 session 2
Reading subject 1 session 3
Reading subject 1 session 4
Reading subject 2 session 1
Reading subject 2 session 2
Reading subject 2 session 3
Reading subject 2 session 4
Reading subject 2 session 5
Reading subject 2 session 6
Reading subject 2 session 7
Reading subject 3 session 1
Reading subject 3 session 2
Reading subject 3 session 3
Reading subject 4 session 1
Reading subject 4 session 2
Reading subject 5 session 1
Reading subject 5 session 2
Reading subject 5 session 3
Reading subject 6 session 1
Reading subject 6 session 2
Reading subject 6 session 3
Reading subject 6 session 4
Reading subject 6 session 5
Reading subject 6 session 6
Reading subject 6 session 7
Reading subject 7 se